In [1]:
import os
import h5py
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm

ns = {'ismrmrd': 'http://www.ismrm.org/ISMRMRD'}

def extract_metadata_from_h5(file_path):
    with h5py.File(file_path, "r") as f:
        try:
            header = f['ismrmrd_header'][()].decode("utf-8")
            root = ET.fromstring(header)
        except Exception:
            return []

        def get(path, default="null"):
            el = root.find(path, ns)
            return el.text if el is not None else default

        num_slices = f['kspace'].shape[0]
        filename = os.path.basename(file_path).replace(".h5", "")

        metadata_rows = []
        for slice_idx in range(num_slices):
            row = {
                "filename": filename,
                "slice": slice_idx,
                "anatomy": "brain" if "brain" in file_path.lower() else "knee",
                "contrast": get(".//ismrmrd:measurementInformation/ismrmrd:protocolName"),
                "sequence": get(".//ismrmrd:sequenceParameters/ismrmrd:sequence_type"),
                "TR": get(".//ismrmrd:sequenceParameters/ismrmrd:TR"),
                "TE": get(".//ismrmrd:sequenceParameters/ismrmrd:TE"),
                "TI": get(".//ismrmrd:sequenceParameters/ismrmrd:TI"),
                "flip_angle": get(".//ismrmrd:sequenceParameters/ismrmrd:flipAngle_deg"),
                "pathology": "null"
            }
            metadata_rows.append(row)
        return metadata_rows

# Batch extract from folder
all_metadata = []
fastmri_dir = ""

for fname in tqdm(os.listdir(os.curdir)):
    if fname.endswith(".h5"):
        full_path = os.path.join(fastmri_dir, fname)
        rows = extract_metadata_from_h5(full_path)
        all_metadata.extend(rows)

df = pd.DataFrame(all_metadata)
df.to_csv("metadata_brain.csv", index=False)

100%|█████████████████████████████████████████| 14/14 [00:00<00:00, 1390.39it/s]


In [2]:
import numpy as np
import os
from tqdm import tqdm
from skimage.transform import resize

def rss_reconstruction(kspace):
    img = np.fft.ifft2(kspace, axes=(-2, -1))
    rss = np.sqrt(np.sum(np.abs(img) ** 2, axis=0))
    return rss

def save_complex_slice_npy(h5_file, output_dir):
    with h5py.File(h5_file, "r") as f:
        kspace = f['kspace'][:]  # (slices, coils, H, W)
        filename = os.path.basename(h5_file).replace(".h5", "")
        anatomy = "brain" if "brain" in h5_file.lower() else "knee"
        
        for i in range(kspace.shape[0]):
            img = np.fft.ifft2(kspace[i], axes=(-2, -1))
            img_sum = np.sum(img, axis=0)  # complex sum-of-coils
            img_resized = resize(img_sum, (320, 320), preserve_range=True, anti_aliasing=True)
            
            out_path = f"./fastmri/{anatomy}_mvue_320_train/slice/{filename}/{i:03d}.npy"
            os.makedirs(os.path.dirname(out_path), exist_ok=True)
            np.save(out_path, img_resized.astype(np.complex64))

# Apply to your .h5 files
for fname in tqdm(os.listdir(os.curdir)):
    if fname.endswith(".h5"):
        save_complex_slice_npy(os.path.join(fastmri_dir, fname), "./fastmri")

  0%|                                                    | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.11/site-packages/skimage/_shared/utils.py:746: ComplexWarning: Casting complex values to real discards the imaginary part
  image = image.astype(float)
100%|███████████████████████████████████████████| 14/14 [00:06<00:00,  2.02it/s]
